In this notebook, we will calculate the "stress of beliefs".

The stress of a belief is the average amount of stress it causes people in the population.

To calculate this, we decompose every person's stress into stress vectors (captured in the heatmat_matrix), and then calculate the average amount of stress that each belief causes. 

We can also sum the negative and positive contribution of each belief.   

In [7]:
""" BORING!!! """
%load_ext autoreload
%autoreload 2
import numpy as np
import pyreadstat as prs
import sys
sys.path.append('../src')
from clean_data_1 import *
from clean_data_2 import transform_dataframe_2
from make_belief_network import make_belief_network
from calc_stress import *
from interpret_belief_vector import interpret_belief_vector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
raw_df, meta = prs.read_sas7bdat("../dat/gss7222_r3.sas7bdat")
df, metadata = transform_dataframe_1(raw_df)    # df contains all our data, metadata contains some other random shit 

In [3]:
variables = ["PARTYID","POLVIEWS","NATSPAC","NATENVIR","NATHEAL","NATCITY","NATCRIME","NATDRUG","NATEDUC","NATRACE","NATARMS",
"NATAID","NATFARE","NATROAD","NATSOC","NATMASS","NATPARK","NATCHLD","NATSCI","EQWLTH","SPKATH","COLATH","LIBATH","SPKRAC","COLRAC","LIBRAC","SPKCOM","COLCOM","LIBCOM","SPKMIL","COLMIL","LIBMIL","SPKHOMO",
"COLHOMO","LIBHOMO","SPKMSLM","COLMSLM","LIBMSLM","CAPPUN","GUNLAW","COURTS","GRASS","ATTEND","RELITEN","POSTLIFE","PRAYER","AFFRMACT","WRKWAYUP","HELPFUL",
"FAIR","TRUST","CONFINAN","CONBUS","CONCLERG","CONEDUC","CONFED","CONLABOR","CONPRESS","CONMEDIC","CONTV","CONJUDGE","CONSCI","CONLEGIS","CONARMY","GETAHEAD","FEPOL","ABDEFECT","ABNOMORE","ABHLTH","ABPOOR","ABRAPE","ABSINGLE","ABANY","SEXEDUC","DIVLAW","PREMARSX","TEENSEX","XMARSEX","HOMOSEX","PORNLAW",
"SPANKING","LETDIE1","SUICIDE1","SUICIDE2","POLHITOK","POLABUSE","POLMURDR","POLESCAP","POLATTAK","NEWS","TVHOURS","FECHLD","FEPRESCH","FEFAM","RACDIF1","RACDIF2","RACDIF3",
"RACDIF4","HELPPOOR","MARHOMO", "PRESLAST_NONCONFORM", "PRESLAST_DEMREP", "VOTELAST", "BALLOT"]

In [9]:
""" In order to calculate the stress of beleif vectors, we need to belief vectors """
""" We first clear the dataset again to implement the median solution """

start_year = 1990
timeframe = list(range(start_year, start_year+4))

BN, variables_list, correlation_matrix_partial = make_belief_network(df, variables, timeframe, method="spearman", is_partial=True, threshold=0, sample_threshold=0, regularisation=0.2)

df_b_vecs, meta_data_minsmaxs = transform_dataframe_2(df, metadata, timeframe)
df_b_vecs = df_b_vecs.fillna(0)
df_b_vecs = normalise_columns(df_b_vecs, meta_data_minsmaxs)
df_b_vecs = df_b_vecs[variables_list]

var_for_split = "BALLOT"

df_ballot_1 = df_b_vecs[df_b_vecs[var_for_split] == 1]
df_ballot_2 = df_b_vecs[df_b_vecs[var_for_split] == 2]
df_ballot_3 = df_b_vecs[df_b_vecs[var_for_split] == 3]
belief_vec_ballot_1 = df_ballot_1.to_numpy()
belief_vec_ballot_2 = df_ballot_2.to_numpy()
belief_vec_ballot_3 = df_ballot_3.to_numpy()

correlation_matrix_noDiag = correlation_matrix_partial - np.eye(correlation_matrix_partial.shape[0])

b1_stress_vec = []
b2_stress_vec = []
b3_stress_vec = []

b1_stress_vec = [stress(belief_vec_ballot_1[i,:], correlation_matrix_noDiag) for i in range(belief_vec_ballot_1.shape[0])]
b2_stress_vec = [stress(belief_vec_ballot_2[i,:], correlation_matrix_noDiag) for i in range(belief_vec_ballot_2.shape[0])]
b3_stress_vec = [stress(belief_vec_ballot_3[i,:], correlation_matrix_noDiag) for i in range(belief_vec_ballot_3.shape[0])]

summed_stress_vec = b1_stress_vec + b2_stress_vec + b3_stress_vec 

In [45]:
""" We can now iterate over all belief vectors and get the summed heatmaps. """
array_of_vectors = belief_vec_ballot_3

heatmap_matrix_sum = np.zeros((len(variables_list), len(variables_list)))
for vector in array_of_vectors:
    stress_decomposition_dict, total_stress = stress_decomposition(vector, correlation_matrix_noDiag, variables_list)
    heatmap_matrix = plot_stress_heatmap(stress_decomposition_dict, variables_list, plot=False)

    heatmap_matrix_sum += heatmap_matrix

heatmap_matrix_avg = heatmap_matrix_sum / len(belief_vec_ballot_1)

In [46]:
""" Now we want to sum down the columns of the summed heatmap matrix (ignoring duplicates as the matrix is symmetrical)"""

belief_stress_contributions = np.triu(heatmap_matrix_avg)
belief_stress_positive_contributions = np.where(belief_stress_contributions > 0, belief_stress_contributions, 0)
belief_stress_negative_contributions = np.where(belief_stress_contributions < 0, belief_stress_contributions, 0)

belief_stress_positive_contributions = np.sum(belief_stress_positive_contributions, axis=0)
belief_stress_negative_contributions = np.sum(belief_stress_negative_contributions, axis=0)
belief_stress_contributions = np.sum(belief_stress_contributions, axis=0)

# Print all beliefs and their total stress contribution, in rank order
for i in np.argsort(belief_stress_contributions):
    print(variables_list[i], belief_stress_contributions[i])

ABANY -0.5127008976187497
LIBHOMO -0.4523180573737218
LIBMIL -0.4442487413392101
ABSINGLE -0.4028089567456235
ABRAPE -0.39935941451763046
COLMIL -0.36821804269475744
LIBCOM -0.36086777248857094
COLHOMO -0.2831696392919024
LIBRAC -0.21113138341539395
COLRAC -0.1967749708766655
SPKMIL -0.19138586818770603
ABPOOR -0.18857544144305022
ABHLTH -0.1797773401860285
PRESLAST_DEMREP -0.1749642021623948
SPKCOM -0.15214967682367303
POLATTAK -0.14238672834163688
SPKHOMO -0.13735453323616165
HOMOSEX -0.1319567267046631
POLMURDR -0.10219803949682718
COLCOM -0.10204464893088169
CONLEGIS -0.09701340836865545
SPKRAC -0.0935938055832271
LIBATH -0.089690598498404
TRUST -0.0878772451453422
XMARSEX -0.0868196242384388
POLESCAP -0.08678314735322284
FAIR -0.07333110127124058
COLATH -0.06604781615172374
CONSCI -0.06569427857363544
RELITEN -0.05423835055915394
HELPPOOR -0.043478088320134056
CONJUDGE -0.041782247900997806
NATDRUG -0.039033819155095045
NATRACE -0.03765167281634087
NATEDUC -0.037513890859154735
CO